In [1]:
import setup

setup.init_django()

In [2]:
from decouple import config

In [3]:
from blog.models import BlogPost
from analytics.models import PageView
from rag import db as rag_db, settings as rag_settings

In [4]:
from sqlalchemy import (
    create_engine,
    inspect,
)

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever

In [5]:
# Initialize the Llama settings
rag_settings.init()

# Get Database pool url from env
vector_database_url = rag_db.get_database_url(use_pooling=False)

In [6]:
engine = create_engine(vector_database_url)

In [7]:
inspect(engine).get_table_names()

['django_migrations',
 'django_content_type',
 'auth_permission',
 'auth_group',
 'auth_group_permissions',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'django_session',
 'blog_blogpost',
 'products_product',
 'products_embedding',
 'analytics_pageview']

In [8]:
tables =[]
models = [BlogPost, PageView]
for model in models:
    table = model._meta.db_table
    tables.append(table)

In [9]:
tables

['blog_blogpost', 'analytics_pageview']

In [10]:
sql_database = SQLDatabase(engine, include_tables=tables)

C:\Users\HP\Documents\Git\my-talk-to-django\venv\Lib\site-packages\llama_index\core\utilities\sql_wrapper.py:112: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [11]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
)

In [12]:
response =  sql_query_engine.query("What blog post has most views?")
print(str(response))

C:\Users\HP\Documents\Git\my-talk-to-django\venv\Lib\site-packages\llama_index\core\utilities\sql_wrapper.py:169: SAWarning: Did not recognize type 'vector' of column 'embedding'
  for column in self._inspector.get_columns(table_name, schema=self._schema):


The blog post with the most views is "Blog Post 3".



In [13]:
for node in response.source_nodes:
    print(node.node.get_content())

[('Blog Post 3',)]


In [14]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=tables, return_raw=True
)

r = nl_sql_retriever.retrieve("What is my least most recent viewed blog post?")

In [15]:
print(r)
for node in r:
    print(node)
    print(node.metadata)

[NodeWithScore(node=TextNode(id_='61ea1f6c-47b4-4109-92aa-0b071969daaa', embedding=None, metadata={'sql_query': 'SELECT\n  bp.title\nFROM analytics_pageview AS ap\nJOIN blog_blogpost AS bp\n  ON ap.post_id = bp.id\nORDER BY\n  ap.timestamp\nLIMIT 1;', 'result': [('Blog Post 1',)], 'col_keys': ['title']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[('Blog Post 1',)]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]
Node ID: 61ea1f6c-47b4-4109-92aa-0b071969daaa
Text: [('Blog Post 1',)]
Score: None

{'sql_query': 'SELECT\n  bp.title\nFROM analytics_pageview AS ap\nJOIN blog_blogpost AS bp\n  ON ap.post_id = bp.id\nORDER BY\n  ap.timestamp\nLIMIT 1;', 'result': [('Blog Post 1',)], 'col_keys': ['title']}
